<a href="https://colab.research.google.com/github/adidror005/youtube-videos/blob/main/Backtesting_py_Actual_Video.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### PART 1 of a many part series on backtesting
1. Backtest optimization
2. Dangers of backtesting
  - Almost all backtests are flawed as a result of selection bias on multiple tests
  - 7 sins of quantiative investing by Deutch bank
3. Backtesting metrics
  - Sharpe ratio
  - Sortino ratio
  - Drawdown
  - Calmer Ratio etc..
4. Connecting with Live Trading
5. Options?

# Backtesting

Backtesting is a historical simulation of how a strategy would have performed.
* What was the sharpe ratio? return? etc..

### Disclaimer: Almost all backtests are flawed.
* As Lopez De Prado writes the author of Advances and Financial Machine Learning puts it in his 2nd Law of backtesting:

"Backtesting while researching is like drinking and driving. Do not research under the influence of a backtest."

* The past could have been different. We just saw one realization
* **Backtest overfitting** from selection bias on multiple tests is extremely easy.
  - Basically you optimize and try a million hyperparams and just select the best ones and don't mention how many different combos you did to get to the best ones!
  - If you have a 1/14 million chance to win the lottery and you keep buying tickets until you win. Once you win you come up with a cool formula for the numbers and you only share the winning formula.
* More suttley the 7 sins of quantitative finance from Deutche Bank can easily flaw your backtest.
  - Survivorship bias
  - Lookahead bias
  - Data Mining+Data Snooping
  - Short selling assumptions
  - transaction costs
  - Storytelling
  - Outliers
* Things like like Margin call stop out assumptions. Especially for brokers like Interactive who will not hesistate to liquidate you at he absolute worst value.


## Goal of Video
* A self containted video showing how to perform a backtest in Python.
* I will try to be as original as possible to compliment other creaters' Youtube videos. Things like trade management i.e.
 adjusting take profit, stop loss, cancelling orders, etc...
* I will go into far greater detail into topics like backtesting metrics, overfitting, and optimization in other videos. I also promise to connect it to live trading.
* If you want me to go into greater depth into any specific topic please write a comment.

## Backtesting in Python
* Tons of backtesting libraries, we will use **backtesting.py** since easiest lightweight to get up and running
* Other popular options:
    - *Vectorbt*: lighting quick numba has good open source version, but they are more focused on their paid pro version which I heard is worth it for what it is worth.
    - *Backtrader*: Very popular + integration with brokers like IBKR, but not actively maintained by original author.
    - Other libraries include PyAlgoTrade as Part Time Larry mentions, but depreciated and now became Basana and Zipline in the now closed Quantopian.
    - *QuantConnect*: People rave about.
    - *NautilusTrader*: event driven backtest and has integration with brokers like IBKR. In rust with Python interface. Seems most natural, but I don't know too much about it.

## Backtesting.py

* Focuses on bar data. Can only do stuff after entire bar.
* Gives you some metrics and beatiful graphs and analytics
https://kernc.github.io/backtesting.py/
* Provides nice way to optimize params and give heatmap
  - backtest.run will run backtests
  - backtest.optimize will optimize backtests
* Since it is bar level there are some drawbacks like say you have a limit buy order and attach a take profit and a stop loss order in the same bar.
* The system won't know what to do!
    - Did the take profit and stop loss happen before or after the limit?
    - Did the take profit happen before the stop loss?

### Installs
* Backtesting.py
* Pandas_Ta

In [1]:
!pip install backtesting
!pip install pandas_ta

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 175.5/175.5 kB 3.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for backtesting: filename=Backtesting-0.3.3-py3-none-any.whl size=173913 sha256=2c64b67d9aedbdcc6d90162a07e1fd32eee770808e0355107d10c9253a904947
  Stored in directory: /root/.cache/pip/wheels/e2/30/7f/19cbe31987c6ebdb47f1f510343249066711609e3da2d57176
Successfully built backtesting
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.1/115.1 kB 2.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pandas_ta: filename=pandas_ta-0.3.14b0-py3-none-any.whl size=218909 sha256=43257e3b45891c6f4c843e86e32dfd72a41bbcb025f08e13ec1cab5e258b086a
  Stored in directory: /root/.cache/pip/wheels/69/00/ac/f7fa862c34b0e2ef320175100c233377b4c558944f12474cf0
Successfully built pandas_ta


### Load 5 min bars in Python for AMD with Pandas

In [3]:
import pandas as pd
df = pd.read_csv('df_5mins.csv',index_col='Date',parse_dates=['Date'])

df

,Open,High,Low,Close,Volume
Date,,,,,
2021-06-07 09:30:00,81.40,81.51,81.09,81.30,1041621.0
2021-06-07 09:35:00,81.26,81.68,81.24,81.26,544794.0
2021-06-07 09:40:00,81.25,81.37,81.04,81.16,660306.0
2021-06-07 09:45:00,81.16,81.29,81.08,81.25,506428.0
2021-06-07 09:50:00,81.25,81.36,80.99,81.07,557572.0
...,...,...,...,...,...
2024-08-15 15:35:00,147.75,147.78,147.53,147.68,643114.0
2024-08-15 15:40:00,147.68,147.96,147.50,147.65,943392.0
2024-08-15 15:45:00,147.64,147.82,147.53,147.56,752626.0


### Teaser Example 1: Simple Oversold RSI strategy with


* Define the strategy

In [4]:
import pandas as pd
import yfinance as yf
from backtesting import Backtest, Strategy
from backtesting.lib import crossover
import pandas_ta as ta

def indicator_rsi(data):
    rsi = ta.rsi(close = data.Close.s,length=14)
    return rsi

# Define strategy
class SimpleRSIStrategy(Strategy):

    def init(self):
        self.rsi = self.I(indicator_rsi,self.data)
        self.OVERSOLD_THRESH = 10
        self.OVERBOUGHT_THRESH = 90
        self.NEUTRAL_THRESH = 50
    def next(self):
        if crossover(self.OVERSOLD_THRESH,self.rsi) and not self.position and not self.orders:
            self.buy(size=100)
        elif crossover(self.rsi,self.OVERBOUGHT_THRESH) and not self.position and not self.orders:
            self.sell(size=100)
        elif crossover(self.rsi,self.OVERBOUGHT_THRESH) and self.position.is_long:
            self.position.close()
        elif crossover(self.rsi,self.OVERSOLD_THRESH) and self.position.is_short:
            self.position.close()

/usr/local/lib/python3.10/dist-packages/backtesting/_plotting.py:50: UserWarning: Jupyter Notebook detected. Setting Bokeh output to notebook. This may not work in Jupyter clients without JavaScript support (e.g. PyCharm, Spyder IDE). Reset with `backtesting.set_bokeh_output(notebook=False)`.
  warnings.warn('Jupyter Notebook detected. '


##### Define the backtest

* cash
* commission: % per trade
* leverage: ...
* slippage: ...



In [8]:
bt = Backtest(data=df, strategy=SimpleRSIStrategy, cash=1000000, commission=.0001)

* Run the backtest

In [9]:
summary = bt.run()


* Look at summary of results

In [10]:
summary

,0
Start,2021-06-07 09:30:00
End,2024-08-15 15:55:00
Duration,1165 days 06:25:00
Exposure Time [%],61.143095
Equity Final [$],1002056.2852
Equity Peak [$],1006081.9686
Return [%],0.205629
Buy & Hold Return [%],81.266913
Return (Ann.) [%],0.064405
Volatility (Ann.) [%],0.517088


#### Backtest Performance Metrics

These metrics evaluate the performance and risk characteristics of a trading strategy over a specified period.

**Performance**

* **Equity Final:** The final value of your investment account after all trading activity.
* **Equity Peak:** The highest value your account reached during the backtest.
* **Return:** The overall percentage gain on your initial investment.
* **Buy & Hold Return:** The return you would have achieved if you simply bought the asset at the start and held it throughout the period.
* **Return (Ann.):** The annualized return, showing the average yearly gain.
* **Volatility (Ann.):** The annualized standard deviation of returns, indicating the degree of price fluctuation.

**Risk**

* **Sharpe Ratio:** Measures risk-adjusted return, comparing excess return to volatility.
* **Sortino Ratio:** Similar to Sharpe Ratio but considers only downside volatility.
* **Calmar Ratio:** Evaluates return relative to the maximum drawdown.
* **Max. Drawdown:** The largest percentage decline from a peak to a subsequent trough in your account value.
* **Avg. Drawdown:** The average percentage decline during drawdown periods.

**Trade Statistics**

* **# Trades:** The total number of trades executed.
* **Win Rate:** The percentage of winning trades.
* **Best Trade:** The highest percentage gain on a single trade.
* **Worst Trade:** The largest percentage loss on a single trade.
* **Avg. Trade:** The average percentage gain/loss per trade.
* **Profit Factor:** The ratio of gross profits to gross losses.
* **Expectancy:** The average expected gain/loss per trade, considering both win rate and average win/loss size.

**Interpretation**

These metrics collectively paint a picture of the strategy's profitability, risk profile, and trade characteristics. A high return with low volatility and drawdowns is ideal. A good win rate, profit factor, and positive expectancy are also desirable trade statistics.

**Remember:** It's important to interpret these metrics in context and consider the specific strategy and market conditions. No single metric tells the whole story; a comprehensive analysis is necessary to evaluate a trading strategy's effectiveness.


* Can look at invididual trades

In [11]:
summary._trades

,Size,EntryBar,ExitBar,EntryPrice,ExitPrice,PnL,ReturnPct,EntryTime,ExitTime,Duration
0,100,5307,12991,103.150314,126.90,2374.9686,0.230243,2021-09-13 09:45:00,2022-02-02 09:35:00,141 days 23:50:00
1,100,16113,20642,113.371336,86.33,-2704.1336,-0.238520,2022-03-31 09:45:00,2022-06-24 10:10:00,85 days 00:25:00
2,-100,24003,26333,97.160283,62.69,3447.0283,0.354778,2022-08-25 10:45:00,2022-10-07 09:55:00,42 days 23:10:00
3,-100,28206,29805,66.003399,68.00,-199.6601,-0.030250,2022-11-10 10:00:00,2022-12-12 09:45:00,31 days 23:45:00
4,-100,29883,30436,75.102489,63.41,1169.2489,0.155687,2022-12-13 09:45:00,2022-12-22 10:20:00,9 days 00:35:00
5,-100,30895,33396,65.853414,78.00,-1214.6586,-0.184449,2023-01-03 09:35:00,2023-02-17 10:00:00,45 days 00:25:00
6,100,37372,37494,82.128212,88.25,612.1788,0.074539,2023-05-03 09:50:00,2023-05-04 13:30:00,1 days 03:40:00
7,-100,38072,42178,99.450054,111.65,-1219.9946,-0.122674,2023-05-16 09:40:00,2023-08-02 10:20:00,78 days 00:40:00
8,100,44594,48332,103.390338,123.54,2014.9662,0.194889,2023-09-15 10:10:00,2023-11-22 09:40:00,67 days 23:30:00
9,-100,48607,49844,124.927506,136.09,-1116.2494,-0.089352,2023-11-29 09:35:00,2023-12-20 15:10:00,21 days 05:35:00


* Can look at plot

In [14]:
# bt.plot(resample=False)

* Be careful if too many points
    - Depending on version you could get bug only thing that worked for me is resample = False...
* In that case you probably should save to file...

In [15]:
bt.plot(resample=False,filename='plot.html')

/usr/local/lib/python3.10/dist-packages/backtesting/_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%d %b'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],
/usr/local/lib/python3.10/dist-packages/backtesting/_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%m/%Y'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],
/usr/local/lib/python3.10/dist-packages/backtesting/_plotting.py:659: UserWarning: found multiple competing values for 'toolbar.active_drag' property; using the latest value
  fig = gridplot(
/usr/local/lib/python3.10/dist-packages/backtesting/_plotting.py:659: UserWarning: found multiple competing values for 'toolbar.active_scroll' property; using the latest value
  fig = gridplot(


GridPlot(id='p1744', ...)

# Main components of backtesting.py

## **Strategy**:
   * Abstract class where we define our trading strategy

#### **Strategy**'s Methods to subclass

* next  
       - When a new bar comes in what is your strategy going to be?
       - For example
           * If SMA_20 crosses above SMA_50 and we have no position then BUY 100 shares
           * If SMA_20 crosses below SMA_50 and we have a position then close position
* init
        - Initialize your indicators and other calculations you need for your strategy
            * i.e. self.sma_20 = ...
            * i.e. self.sma_50 = ...
        - Basically calculate them vectorized for performance

## **Special Class Variables**:

* *self.data* and *self.I*:

**BIG PICTURE**: For next() gives you the latest value 1 at a time for each bar.

* self.data:
        - Backtestings's data class sort of like a dataframe, but not exactly for "optimization purposes".
        - It is numpy arrays, but has OHLCV moethods.
        - in the next() method: it only reveals the latest value 1 at a time similar to how you would have in real life
* self.I
        - self.I(some_function,data):
        - *some_function* can be any function i.e. technical indicators
            - RSI
            - MACD
        - In the init(): it calculates any indicator for every bar in the dataset in the init, but
        - in the next(): it only reveals the latest value 1 at a time similar to how you would have in real life
      
* NOTE next() only starts after you have at least 2 bars without any NAs.
            - So for an indicator based on 20, 30,and 50 bars the first next() has 52 elements, 2nd next() has 53, and so on.
            - If your indicator doesn't have NAs say like High/Low or something simple then first next() has 2 elements, 2nd next() 3, and so on.
            -So the first next() would have 20 elements then 21 and so on starting with first time

* Kinda helps prevent lookahead bias but not really


# Example Build 1 step at a time
Let's illustrate these concepts and buld an example 1 step at a time
* Dummy do nothing strategy
* Real strategy
* Ordering + other important variables
* Order management (cancelling/changing TP and so on)

In [16]:
import numpy as np
def dummy_indicator(data):
  return np.arange(len(data))

class MyDoNothingStrategy(Strategy):
  def init(self):
    self.dummy_indicator = dummy_indicator(self.data)#self.I(dummy_indicator,self.data)
  def next(self):
    print("Dummy Indicator")
    print(self.dummy_indicator)
    print("Data.Close")
    print(self.data.Close)
    print("Data.index")
    print(self.data.index)
bt = Backtest(df.head(10),MyDoNothingStrategy, cash=1000000, commission=.0001)
bt.run()

Dummy Indicator
[0 1 2 3 4 5 6 7 8 9]
Data.Close
[81.3  81.26]
Data.index
DatetimeIndex(['2021-06-07 09:30:00', '2021-06-07 09:35:00'], dtype='datetime64[ns]', freq=None)
Dummy Indicator
[0 1 2 3 4 5 6 7 8 9]
Data.Close
[81.3  81.26 81.16]
Data.index
DatetimeIndex(['2021-06-07 09:30:00', '2021-06-07 09:35:00',
               '2021-06-07 09:40:00'],
              dtype='datetime64[ns]', freq=None)
Dummy Indicator
[0 1 2 3 4 5 6 7 8 9]
Data.Close
[81.3  81.26 81.16 81.25]
Data.index
DatetimeIndex(['2021-06-07 09:30:00', '2021-06-07 09:35:00',
               '2021-06-07 09:40:00', '2021-06-07 09:45:00'],
              dtype='datetime64[ns]', freq=None)
Dummy Indicator
[0 1 2 3 4 5 6 7 8 9]
Data.Close
[81.3  81.26 81.16 81.25 81.07]
Data.index
DatetimeIndex(['2021-06-07 09:30:00', '2021-06-07 09:35:00',
               '2021-06-07 09:40:00', '2021-06-07 09:45:00',
               '2021-06-07 09:50:00'],
              dtype='datetime64[ns]', freq=None)
Dummy Indicator
[0 1 2 3 4 5 6 7 8 9]
Da

,0
Start,2021-06-07 09:30:00
End,2021-06-07 10:15:00
Duration,0 days 00:45:00
Exposure Time [%],0.0
Equity Final [$],1000000.0
Equity Peak [$],1000000.0
Return [%],0.0
Buy & Hold Return [%],-0.221402
Return (Ann.) [%],0.0
Volatility (Ann.) [%],NaN


* Using self.I reveals elements 1 thing at a time

In [18]:
import numpy as np
def dummy_indicator(data):
  return np.arange(len(data))

class MyDoNothingStrategy(Strategy):
  def init(self):
    self.dummy_indicator = self.I(dummy_indicator,self.data)
  def next(self):
    print("Dummy Indicator")
    print(self.dummy_indicator)
    print("Dummy Indicator Last Point")
    print(self.dummy_indicator[-1])
    #print("Data.Close")
    #print(self.data.Close)
    #print("Data.index")
    #print(self.data.index)
bt = Backtest(df.head(10),MyDoNothingStrategy, cash=1000000, commission=.0001)
bt.run()

Dummy Indicator
[0 1]
Dummy Indicator Last Point
1
Dummy Indicator
[0 1 2]
Dummy Indicator Last Point
2
Dummy Indicator
[0 1 2 3]
Dummy Indicator Last Point
3
Dummy Indicator
[0 1 2 3 4]
Dummy Indicator Last Point
4
Dummy Indicator
[0 1 2 3 4 5]
Dummy Indicator Last Point
5
Dummy Indicator
[0 1 2 3 4 5 6]
Dummy Indicator Last Point
6
Dummy Indicator
[0 1 2 3 4 5 6 7]
Dummy Indicator Last Point
7
Dummy Indicator
[0 1 2 3 4 5 6 7 8]
Dummy Indicator Last Point
8
Dummy Indicator
[0 1 2 3 4 5 6 7 8 9]
Dummy Indicator Last Point
9


,0
Start,2021-06-07 09:30:00
End,2021-06-07 10:15:00
Duration,0 days 00:45:00
Exposure Time [%],0.0
Equity Final [$],1000000.0
Equity Peak [$],1000000.0
Return [%],0.0
Buy & Hold Return [%],-0.221402
Return (Ann.) [%],0.0
Volatility (Ann.) [%],NaN


* Indicators with more than 1 dimension example
  - Use .s to turn Close of backtesting self.data type into pandas Series so we can use for examples pandas_ta
  - Use .df to turn into dataframe...
- Recall only starts when at least 2 non-missing values

In [22]:
def bbands_indicator(data):
  bbands = ta.bbands(close=data.Close,length=5).iloc[:,0:3]
  return bbands

In [23]:
bbands_indicator(df.head(10))

,BBL_5_2.0,BBM_5_2.0,BBU_5_2.0
2021-06-07 09:30:00,NaN,NaN,NaN
2021-06-07 09:35:00,NaN,NaN,NaN
2021-06-07 09:40:00,NaN,NaN,NaN
2021-06-07 09:45:00,NaN,NaN,NaN
2021-06-07 09:50:00,81.042398,81.208,81.373602
2021-06-07 09:55:00,80.980462,81.240,81.499538
2021-06-07 10:00:00,80.983366,81.258,81.532634
2021-06-07 10:05:00,81.007926,81.332,81.656074
2021-06-07 10:10:00,81.045037,81.386,81.726963
2021-06-07 10:15:00,81.091737,81.396,81.700263


* Important since using backtestin.py's data type you need the .s method to convert data.Close to pandas seris

In [28]:
def bbands_indicator(data):
  bbands = ta.bbands(close=data.Close.s,length=5).iloc[:,0:3]
  return bbands

class MyDoNothingStrategy(Strategy):

  def init(self):
    self.bbands = self.I(bbands_indicator,self.data)
    self.lower_band = self.bbands[0]
    self.middle_band = self.bbands[1]
    self.upper_band = self.bbands[2]
    #print(self.bbands)
  def next(self):
    print("Lower Band")
    print(self.lower_band)
    print("Middle Band")
    print(self.middle_band)
    print("Upper Band")
    print(self.upper_band)

bt = Backtest(df.head(10),MyDoNothingStrategy, cash=1000000, commission=.0001)
bt.run()

Lower Band
[        nan         nan         nan         nan 81.04239807 80.98046195]
Middle Band
[   nan    nan    nan    nan 81.208 81.24 ]
Upper Band
[        nan         nan         nan         nan 81.37360193 81.49953805]
Lower Band
[        nan         nan         nan         nan 81.04239807 80.98046195
 80.9833657 ]
Middle Band
[   nan    nan    nan    nan 81.208 81.24  81.258]
Upper Band
[        nan         nan         nan         nan 81.37360193 81.49953805
 81.5326343 ]
Lower Band
[        nan         nan         nan         nan 81.04239807 80.98046195
 80.9833657  81.00792593]
Middle Band
[   nan    nan    nan    nan 81.208 81.24  81.258 81.332]
Upper Band
[        nan         nan         nan         nan 81.37360193 81.49953805
 81.5326343  81.65607407]
Lower Band
[        nan         nan         nan         nan 81.04239807 80.98046195
 80.9833657  81.00792593 81.04503666]
Middle Band
[   nan    nan    nan    nan 81.208 81.24  81.258 81.332 81.386]
Upper Band
[        nan   

,0
Start,2021-06-07 09:30:00
End,2021-06-07 10:15:00
Duration,0 days 00:45:00
Exposure Time [%],0.0
Equity Final [$],1000000.0
Equity Peak [$],1000000.0
Return [%],0.0
Buy & Hold Return [%],-0.221402
Return (Ann.) [%],0.0
Volatility (Ann.) [%],NaN


## Let's create strategy **RSIBbandStrategy**
* SHORT IF OVERBOUGHT AND ABOVE UPPER BAND
* LONG IF OVERSOLD AND BELOW LOWER BAND
* CLOSE POSITION IN MIDDLE BAND

#### Init part
* Compute BBAND + RSI indicators vectorized for computation efficiency
* Wrap in self.I so we get it one at a time...

In [29]:
def rsi_indicator(data):
  return ta.rsi(close=data.Close.s,length=14)
def bbands_indicator(data):
  bbands = ta.bbands(close=data.Close.s,length=20).iloc[:,0:3]
  return bbands

class RSIBbandStrategy(Strategy):
  OVERSOLD_THRESH = 30
  OVERBOUGHT_THRESH = 70
  NEUTRAL_THRESH = 50

  def init(self):
    self.bbands = self.I(bbands_indicator,self.data)
    self.lower_band = self.bbands[0]
    self.middle_band = self.bbands[1]
    self.upper_band = self.bbands[2]
    self.rsi = self.I(rsi_indicator,self.data)
    print("BBANDS SHAPE")
    print(self.bbands.shape)
    print("RSI SHAPE")
    print(self.rsi.shape)
  def next(self):
    pass

bt = Backtest(df.head(10000),RSIBbandStrategy, cash=1000000, commission=.0001)
bt.run()




BBANDS SHAPE
(3, 10000)
RSI SHAPE
(10000,)


,0
Start,2021-06-07 09:30:00
End,2021-12-07 13:45:00
Duration,183 days 04:15:00
Exposure Time [%],0.0
Equity Final [$],1000000.0
Equity Peak [$],1000000.0
Return [%],0.0
Buy & Hold Return [%],78.99139
Return (Ann.) [%],0.0
Volatility (Ann.) [%],0.0


### Add our trading strategy logic
* next part()

In [30]:
def rsi_indicator(data):
  return ta.rsi(close=data.Close.s,length=14)
def bbands_indicator(data):
  bbands = ta.bbands(close=data.Close.s,length=20).iloc[:,0:3]
  return bbands

class RSIBbandStrategy(Strategy):
  OVERSOLD_THRESH = 30
  OVERBOUGHT_THRESH = 70
  NEUTRAL_THRESH = 50

  def init(self):
    self.bbands = self.I(bbands_indicator,self.data)
    self.lower_band = self.bbands[0]
    self.middle_band = self.bbands[1]
    self.upper_band = self.bbands[2]
    self.rsi = self.I(rsi_indicator,self.data)
    print("BBANDS SHAPE")
    print(self.bbands.shape)
    print("RSI SHAPE")
    print(self.rsi.shape)
  def next(self):
    if self.OVERSOLD_THRESH>=self.rsi[-1] and not self.position and self.lower_band[-1]<=self.data.Close[-1]:
      self.buy(size=100)
    elif self.rsi[-1]>=self.OVERBOUGHT_THRESH and not self.position and self.data.Close[-1]>=self.upper_band[-1]:
      self.sell(size=100)
    elif self.position.is_long and self.data.Close[-1]>=self.middle_band[-1]:
      self.position.close()
    elif self.position.is_short and self.data.Close[-1]<=self.middle_band[-1]:
      self.position.close()

bt = Backtest(df,RSIBbandStrategy, cash=1000000, commission=.0001)
bt.run()




BBANDS SHAPE
(3, 62532)
RSI SHAPE
(62532,)


,0
Start,2021-06-07 09:30:00
End,2024-08-15 15:55:00
Duration,1165 days 06:25:00
Exposure Time [%],23.653489
Equity Final [$],982456.5756
Equity Peak [$],1000163.1687
Return [%],-1.754342
Buy & Hold Return [%],81.266913
Return (Ann.) [%],-0.554441
Volatility (Ann.) [%],0.303659


* Note: maybe it would be more proper to use the crossover method which
under the hood does...

In [31]:
??crossover

In [33]:
def rsi_indicator(data):
  return ta.rsi(close=data.Close.s,length=14)
def bbands_indicator(data):
  bbands = ta.bbands(close=data.Close.s,length=20).iloc[:,0:3]
  return bbands

class RSIBbandStrategy(Strategy):
  OVERSOLD_THRESH = 30
  OVERBOUGHT_THRESH = 70
  NEUTRAL_THRESH = 50

  def init(self):
    self.bbands = self.I(bbands_indicator,self.data)
    self.lower_band = self.bbands[0]
    self.middle_band = self.bbands[1]
    self.upper_band = self.bbands[2]
    self.rsi = self.I(rsi_indicator,self.data)
    print("BBANDS SHAPE")
    print(self.bbands.shape)
    print("RSI SHAPE")
    print(self.rsi.shape)
  def next(self):
    if crossover(self.OVERSOLD_THRESH,self.rsi) and not self.position and crossover(self.lower_band,self.data.Close):
      self.buy(size=100)
    elif crossover(self.rsi,self.OVERBOUGHT_THRESH) and not self.position and crossover(self.data.Close,self.upper_band):
      self.sell(size=100)
    elif self.position.is_long and crossover(self.data.Close,self.middle_band):
      self.position.close()
    elif self.position.is_short and crossover(self.data.Close,self.middle_band):
      self.position.close()

bt = Backtest(df,RSIBbandStrategy, cash=1000000, commission=.0001)
bt.run()




BBANDS SHAPE
(3, 62532)
RSI SHAPE
(62532,)


,0
Start,2021-06-07 09:30:00
End,2024-08-15 15:55:00
Duration,1165 days 06:25:00
Exposure Time [%],21.139577
Equity Final [$],991102.9874
Equity Peak [$],1000538.0814
Return [%],-0.889701
Buy & Hold Return [%],81.266913
Return (Ann.) [%],-0.280036
Volatility (Ann.) [%],0.302525


## Ordering
* self.buy/self.sell
  - limit (if don't include it is just a market)
      - NOTE IN THIS CASE logic like self.orders is None is important to prevent overbuying
  - tp: take profit (default None)
  - sl: stop loss (deafault None)
  * Note by default market orders occur at next bar's open
  * If next bar has high>tp and low < sl it takes the worst approach.
  * Can use a much smaller time frame for better accuracy.

## Other important Variables
* self.position stores the position
    - Close position with self.position.close()
      - Very useful to prevent over selling like say selling 100 twice when have position of 100 and now you become net short
    - If Net long self.position.is_long = True
    - If Net short self.position.is_short = True
    - self.position.size() is postion size
  

### Same example + adding Limit, TP, and SL
  * Limit at close
  * tp at close + 1
  * sl at close -1

**NOTE** very important to check self.orders to not double orders. i.e. if self.orders

In [34]:
def rsi_indicator(data):
  return ta.rsi(close=data.Close.s,length=14)
def bbands_indicator(data):
  bbands = ta.bbands(close=data.Close.s,length=20).iloc[:,0:3]
  return bbands

class RSIBbandStrategy(Strategy):
  OVERSOLD_THRESH = 30
  OVERBOUGHT_THRESH = 70
  NEUTRAL_THRESH = 50

  def init(self):
    self.bbands = self.I(bbands_indicator,self.data)
    self.lower_band = self.bbands[0]
    self.middle_band = self.bbands[1]
    self.upper_band = self.bbands[2]
    self.rsi = self.I(rsi_indicator,self.data)
    print("BBANDS SHAPE")
    print(self.bbands.shape)
    print("RSI SHAPE")
    print(self.rsi.shape)
  def next(self):
    if crossover(self.OVERSOLD_THRESH,self.rsi) and not self.position and crossover(self.lower_band,self.data.Close) and not self.orders:
      self.buy(size=100,limit=self.data.Close[-1],tp=self.data.Close[-1]+1,sl=self.data.Close[-1]-1)
    elif crossover(self.rsi,self.OVERBOUGHT_THRESH) and not self.position and crossover(self.data.Close,self.upper_band) and not self.orders:
      self.sell(size=100,limit=self.data.Close[-1],tp=self.data.Close[-1]-1,sl=self.data.Close[-1]+1)
    elif self.position.is_long and crossover(self.data.Close,self.middle_band):
      self.position.close()
    elif self.position.is_short and crossover(self.data.Close,self.middle_band):
      self.position.close()

bt = Backtest(df,RSIBbandStrategy, cash=1000000, commission=.0001)
bt.run()




BBANDS SHAPE
(3, 62532)
RSI SHAPE
(62532,)


/usr/local/lib/python3.10/dist-packages/backtesting/backtesting.py:912: UserWarning: (2021-07-28 09:35:00) A contingent SL/TP order would execute in the same bar its parent stop/limit order was turned into a trade. Since we can't assert the precise intra-candle price movement, the affected SL/TP order will instead be executed on the next (matching) price/bar, making the result (of this trade) somewhat dubious. See https://github.com/kernc/backtesting.py/issues/119
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/backtesting/backtesting.py:912: UserWarning: (2021-08-04 09:35:00) A contingent SL/TP order would execute in the same bar its parent stop/limit order was turned into a trade. Since we can't assert the precise intra-candle price movement, the affected SL/TP order will instead be executed on the next (matching) price/bar, making the result (of this trade) somewhat dubious. See https://github.com/kernc/backtesting.py/issues/119
  warnings.warn(
/usr/local/lib/python3.10/di

,0
Start,2021-06-07 09:30:00
End,2024-08-15 15:55:00
Duration,1165 days 06:25:00
Exposure Time [%],7.337043
Equity Final [$],995146.5956
Equity Peak [$],1000802.0109
Return [%],-0.48534
Buy & Hold Return [%],81.266913
Return (Ann.) [%],-0.152695
Volatility (Ann.) [%],0.167531


### Another example: Sell for at least \$1 more
* Let's add a limit order
  - Very important to check that you don't have existing orders!
* Let's remove shorting
* Let's only sell if we are up at least 1\$ per share (100$)





In [40]:
def rsi_indicator(data):
  return ta.rsi(close=data.Close.s,length=14)
def bbands_indicator(data):
  bbands = ta.bbands(close=data.Close.s,length=20).iloc[:,0:3]
  return bbands

class RSIBbandStrategy(Strategy):
  OVERSOLD_THRESH = 30
  OVERBOUGHT_THRESH = 70
  NEUTRAL_THRESH = 50

  def init(self):
    self.last_buy_price = None
    self.bbands = self.I(bbands_indicator,self.data)
    self.lower_band = self.bbands[0]
    self.middle_band = self.bbands[1]
    self.upper_band = self.bbands[2]
    self.rsi = self.I(rsi_indicator,self.data)

  def next(self):
    if crossover(self.OVERSOLD_THRESH,self.rsi) and not self.position and crossover(self.lower_band,self.data.Close) and not self.orders:
      self.buy_price = self.data.Close[-1]
      self.buy(size=100,limit=self.buy_price,tp=self.buy_price+1)
    elif self.position.is_long and crossover(self.data.Close,self.middle_band) and not self.orders:
      self.sell(size=100,limit=self.buy_price+1)

bt = Backtest(df,RSIBbandStrategy, cash=1000000, commission=.0001)

summary = bt.run()




/usr/local/lib/python3.10/dist-packages/backtesting/backtesting.py:912: UserWarning: (2021-11-10 09:35:00) A contingent SL/TP order would execute in the same bar its parent stop/limit order was turned into a trade. Since we can't assert the precise intra-candle price movement, the affected SL/TP order will instead be executed on the next (matching) price/bar, making the result (of this trade) somewhat dubious. See https://github.com/kernc/backtesting.py/issues/119
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/backtesting/backtesting.py:912: UserWarning: (2022-02-03 09:35:00) A contingent SL/TP order would execute in the same bar its parent stop/limit order was turned into a trade. Since we can't assert the precise intra-candle price movement, the affected SL/TP order will instead be executed on the next (matching) price/bar, making the result (of this trade) somewhat dubious. See https://github.com/kernc/backtesting.py/issues/119
  warnings.warn(
/usr/local/lib/python3.10/di

In [41]:
summary._trades

,Size,EntryBar,ExitBar,EntryPrice,ExitPrice,PnL,ReturnPct,EntryTime,ExitTime,Duration
0,100,395,432,80.368036,81.36,99.1964,0.012343,2021-06-14 09:55:00,2021-06-14 13:00:00,0 days 03:05:00
1,100,601,624,79.607960,80.80,119.2040,0.014974,2021-06-16 14:05:00,2021-06-17 09:30:00,0 days 19:25:00
2,100,781,942,83.088308,84.08,99.1692,0.011935,2021-06-21 09:35:00,2021-06-23 10:00:00,2 days 00:25:00
3,100,1483,1490,92.459245,93.45,99.0755,0.010716,2021-07-02 09:35:00,2021-07-02 10:10:00,0 days 00:35:00
4,100,1678,2637,91.319131,92.31,99.0869,0.010851,2021-07-07 12:50:00,2021-07-23 14:45:00,16 days 01:55:00
...,...,...,...,...,...,...,...,...,...,...
67,100,52883,53053,173.997398,175.00,100.2602,0.005762,2024-02-16 14:55:00,2024-02-22 09:35:00,5 days 18:40:00
68,100,53365,53386,175.187517,176.18,99.2483,0.005665,2024-02-28 09:35:00,2024-02-28 11:20:00,0 days 01:45:00
69,100,53989,53990,198.019800,202.72,470.0200,0.023736,2024-03-11 09:35:00,2024-03-11 09:40:00,0 days 00:05:00
70,100,53993,53994,197.979796,198.99,101.0204,0.005103,2024-03-11 09:55:00,2024-03-11 10:00:00,0 days 00:05:00


### Trade Management

* ***self.orders*** see the orders that haven't been filled yet. You can cancel orders by doing for order in self.orders: order.cancel_order().
  - For modifying not yet filled trades they recommend cancelling and then adjusting limit,tp and stop loss
* ***self.trades***: Can access not yet finished round trip trades
  - For example say you buy 100 you are net long this will still give you a trade with size 100 and if you close position it will remove. If you sell 20 shares then it will have trade with size 20.
* ***self.closed_trades***: Is entire roundtrip so it only happens if you open a  position and then close it. For partial closes like say you buy 100 and sell 40 it would show the completed trades at 40
* Can modify stop losses and take profits by editing the trade.tp and trade in *self.trades*
* NOTE contingent means some sort of parent/child relationship

In [42]:
from itertools import filterfalse
import numpy as np
from backtesting._util  import _Data
# .s makes it a series
def bbands_indicator(data):
  bbands = ta.bbands(close=data.Close.s,length=20).iloc[:,0:3]
  return bbands
def rsi_indicator(data):
  rsi_indicator = ta.rsi(close=data.Close.s,length=14)
  return rsi_indicator

class MyExampleTradeManagementStrategy(Strategy):

  def init(self):
    self.bbands = self.I(bbands_indicator,self.data)
    self.upper_band = self.bbands[0]
    self.lower_band = self.bbands[2]
    self.middle_band = self.bbands[1]
    self.rsi = self.I(rsi_indicator,self.data)
    self.OVERSOLD_THRESH = 10
    self.OVERBOUGHT_THRESH = 90
    self.NEUTRAL_THRESH = 50
    print(self.lower_band[-100:])
    self.continue_printing = True
  def next(self):

    if self.continue_printing:
      print("START NEW BAR")
      print("Orders")
      print(self.orders)
      print("Trades")
      print(self.trades)
      print("Close Trades")
      print(self.closed_trades)
      print("Position")
      print(self.position)

    if len(self.closed_trades)>0:
      self.continue_printing = False
    elif self.rsi[-1]<=self.OVERSOLD_THRESH and not self.position and self.data.Close[-1]<=self.lower_band[-1]:
        self.buy(size=100)
        if self.continue_printing:
          print("PLACED ORDER")
          print("Orders")
          print(self.orders)
          print("Trades")
          print(self.trades)
          print("END CURRENT BAR")
    elif self.position.is_long and self.data.Close[-1]>=self.middle_band[-1]:
        self.position.close()
        print("CLOSING POSITION")
        print("Orders")
        print(self.orders)
        print("Trades")
        print(self.trades)
        print("Position")
        print(self.position)
        print("END CURRENT BAR")





bt = Backtest(df.head(10000),MyExampleTradeManagementStrategy, cash=1000000, commission=.0001)

bt.run()

Streaming output truncated to the last 5000 lines.
START NEW BAR
Orders
()
Trades
()
Close Trades
()
Position
<Position: 0 (0 trades)>
START NEW BAR
Orders
()
Trades
()
Close Trades
()
Position
<Position: 0 (0 trades)>
START NEW BAR
Orders
()
Trades
()
Close Trades
()
Position
<Position: 0 (0 trades)>
START NEW BAR
Orders
()
Trades
()
Close Trades
()
Position
<Position: 0 (0 trades)>
START NEW BAR
Orders
()
Trades
()
Close Trades
()
Position
<Position: 0 (0 trades)>
START NEW BAR
Orders
()
Trades
()
Close Trades
()
Position
<Position: 0 (0 trades)>
START NEW BAR
Orders
()
Trades
()
Close Trades
()
Position
<Position: 0 (0 trades)>
START NEW BAR
Orders
()
Trades
()
Close Trades
()
Position
<Position: 0 (0 trades)>
START NEW BAR
Orders
()
Trades
()
Close Trades
()
Position
<Position: 0 (0 trades)>
START NEW BAR
Orders
()
Trades
()
Close Trades
()
Position
<Position: 0 (0 trades)>
START NEW BAR
Orders
()
Trades
()
Close Trades
()
Position
<Position: 0 (0 trades)>
START NEW BAR
Orders
()
T

/usr/local/lib/python3.10/dist-packages/backtesting/_stats.py:122: RuntimeWarning: divide by zero encountered in scalar divide
  s.loc['Sortino Ratio'] = np.clip((annualized_return - risk_free_rate) / (np.sqrt(np.mean(day_returns.clip(-np.inf, 0)**2)) * np.sqrt(annual_trading_days)), 0, np.inf)  # noqa: E501


,0
Start,2021-06-07 09:30:00
End,2021-12-07 13:45:00
Duration,183 days 04:15:00
Exposure Time [%],0.16
Equity Final [$],1000073.9686
Equity Peak [$],1000073.9686
Return [%],0.007397
Buy & Hold Return [%],78.99139
Return (Ann.) [%],0.01445
Volatility (Ann.) [%],0.01038


* What about for partial orders?

In [43]:
from itertools import filterfalse
import numpy as np
from backtesting._util  import _Data
# .s makes it a series
def bbands_indicator(data):
  bbands = ta.bbands(close=data.Close.s,length=20).iloc[:,0:3]
  return bbands
def rsi_indicator(data):
  rsi_indicator = ta.rsi(close=data.Close.s,length=14)
  return rsi_indicator

class MyPartialOrdersExampleStrategy(Strategy):

  def init(self):
    self.bbands = self.I(bbands_indicator,self.data)
    self.upper_band = self.bbands[0]
    self.lower_band = self.bbands[2]
    self.middle_band = self.bbands[1]
    self.rsi = self.I(rsi_indicator,self.data)
    self.OVERSOLD_THRESH = 30
    self.OVERBOUGHT_THRESH = 70
    self.NEUTRAL_THRESH = 50
    self.continue_printing = True
    self.stop_printing = False
  def next(self):

    if self.continue_printing:
      print("Orders")
      print(self.orders)
      print("Trades")
      print(self.trades)
      print("Close Trades")
      print(self.closed_trades)
      print("Position")
      print(self.position)

    if self.stop_printing:
      self.continue_printing = False

    if len(self.closed_trades)==2:
      self.continue_printing = False
    elif self.rsi[-1]<=self.OVERSOLD_THRESH and not self.position and len(self.orders)==0 and self.data.Close[-1]<=self.lower_band[-1]:
        self.buy(size=100,limit=self.data.Close[-1]-0.3)
    elif self.position.is_long and self.data.Close[-1]>=self.middle_band[-1]:
        self.sell(size=10)
        if self.position.size==90:
          self.stop_printing = True


bt = Backtest(df.head(10000),MyPartialOrdersExampleStrategy, cash=1000000, commission=.0001)

sumary = bt.run()

Orders
()
Trades
()
Close Trades
()
Position
<Position: 0 (0 trades)>
Orders
()
Trades
()
Close Trades
()
Position
<Position: 0 (0 trades)>
Orders
()
Trades
()
Close Trades
()
Position
<Position: 0 (0 trades)>
Orders
(<Order size=100.0, limit=80.33, contingent=0>,)
Trades
()
Close Trades
()
Position
<Position: 0 (0 trades)>
Orders
(<Order size=100.0, limit=80.33, contingent=0>,)
Trades
()
Close Trades
()
Position
<Position: 0 (0 trades)>
Orders
(<Order size=100.0, limit=80.33, contingent=0>,)
Trades
()
Close Trades
()
Position
<Position: 0 (0 trades)>
Orders
(<Order size=100.0, limit=80.33, contingent=0>,)
Trades
()
Close Trades
()
Position
<Position: 0 (0 trades)>
Orders
(<Order size=100.0, limit=80.33, contingent=0>,)
Trades
()
Close Trades
()
Position
<Position: 0 (0 trades)>
Orders
(<Order size=100.0, limit=80.33, contingent=0>,)
Trades
()
Close Trades
()
Position
<Position: 0 (0 trades)>
Orders
(<Order size=100.0, limit=80.33, contingent=0>,)
Trades
()
Close Trades
()
Position
<Po

### Order Cancelling Example
*  Example where we immediately cancel an order next bar after we place it and place limit order for 2$ less if random number <=0.1
* By defaut it is like tif = gtc, but as an exercise try to do tif = 'day'

In [46]:
np.random.seed(1)
class CancelOrdersStrategy(Strategy):
  def init(self):
    self.just_bought = False
  def next(self):
    if self.just_bought:
      print("cancelling orders")
      for order in self.orders:
        order.cancel()

    if np.random.random()<0.1:
      self.buy(size=100,limit=self.data.Close[-1]-2)
      self.just_bought=True
    else:
      self.just_bought=False
    # To get 0 trades we cancel immediately
    #for order in self.orders:
     #   order.cancel()

bt = Backtest(df,CancelOrdersStrategy, cash=1000000, commission=.0001)
bt.run()

Streaming output truncated to the last 5000 lines.
cancelling orders
cancelling orders
cancelling orders
cancelling orders
cancelling orders
cancelling orders
cancelling orders
cancelling orders
cancelling orders
cancelling orders
cancelling orders
cancelling orders
cancelling orders
cancelling orders
cancelling orders
cancelling orders
cancelling orders
cancelling orders
cancelling orders
cancelling orders
cancelling orders
cancelling orders
cancelling orders
cancelling orders
cancelling orders
cancelling orders
cancelling orders
cancelling orders
cancelling orders
cancelling orders
cancelling orders
cancelling orders
cancelling orders
cancelling orders
cancelling orders
cancelling orders
cancelling orders
cancelling orders
cancelling orders
cancelling orders
cancelling orders
cancelling orders
cancelling orders
cancelling orders
cancelling orders
cancelling orders
cancelling orders
cancelling orders
cancelling orders
cancelling orders
cancelling orders
cancelling orders
cancelling or

,0
Start,2021-06-07 09:30:00
End,2024-08-15 15:55:00
Duration,1165 days 06:25:00
Exposure Time [%],97.255805
Equity Final [$],1057959.5807
Equity Peak [$],1216055.5822
Return [%],5.795958
Buy & Hold Return [%],81.266913
Return (Ann.) [%],1.781632
Volatility (Ann.) [%],8.658004


* Note still 28 trades since sometimes they were already filled since next bar low<close-2...
* If you cancel the order immediately after placing it then it automatically cancels in this framework...
* Without cancelling

In [47]:
np.random.seed(1)
class DontCancelOrdersStrategy(Strategy):
  def init(self):
    self.just_bought = False
  def next(self):
    #if self.just_bought:
      #print("cancelling orders")
      #for order in self.orders:
       # order.cancel()

    if np.random.random()<0.1:
      self.buy(size=100,limit=self.data.Close[-1]-2)
      self.just_bought=True
    else:
      self.just_bought=False
    # To get 0 trades we cancel immediately
    #for order in self.orders:
     #   order.cancel()

bt = Backtest(df,DontCancelOrdersStrategy, cash=1000000, commission=.0001)
bt.run()

,0
Start,2021-06-07 09:30:00
End,2024-08-15 15:55:00
Duration,1165 days 06:25:00
Exposure Time [%],99.040491
Equity Final [$],1693695.088
Equity Peak [$],2604955.088
Return [%],69.369509
Buy & Hold Return [%],81.266913
Return (Ann.) [%],17.957205
Volatility (Ann.) [%],64.528123


### Changing the take profit/stop loss Example

In [52]:
class ChangeTPSLStrategy(Strategy):
  def init(self):
    self.ordered = False
    self.just_ordered = True
  def next(self):

    if self.just_ordered:
      print("Change TP/SL")
      for trade in self.trades:
        trade.tp=100000
        trade.sl=0.001


    if not self.ordered:
      self.buy(size=100,tp=self.data.Close[-1]+15,sl=self.data.Close[-1]-15)
      self.just_ordered = True
      self.ordered = True
    else:
      self.just_ordered = False

bt = Backtest(df,ChangeTPSLStrategy, cash=1000000, commission=.0001)
summary=bt.run()

Change TP/SL
Change TP/SL


In [53]:
summary._trades

,Size,EntryBar,ExitBar,EntryPrice,ExitPrice,PnL,ReturnPct,EntryTime,ExitTime,Duration
0,100,2,62531,81.258125,147.21,6595.1875,0.811634,2021-06-07 09:40:00,2024-08-15 15:55:00,1165 days 06:15:00


# Next Video
####  More advanced things in next video
* Can do more advanced things like lower our average, position strategies etc..
* Optimize Parameters
* Walk forward method
* Dangers of backtesting
* Create a 1-1 mapping with IBKR trading
* Open to suggestions

### IF YOU ENJOY PLEASE LIKE AND SUBSCRIBE!!! <3
